# Watson Studioでscikit-learn機械学習モデルをWebサービス化する

### 目次

[2. データロードと学習](#ch_2)  
  [2.1 データのロード](#sec_2_1)  
  [2.2 学習](#sec_2_2)  

[3. モデルの保存](#ch_3)  
  [3.1 必要ライブラリの導入](#sec_3_1)  
  [3.2 APIClient インスタンスの生成とデプロイメントスペースの関連付け](#sec_3_2)  
  [3.3 Software Specification ID の取得](#sec_3_3)  
  [3.4 モデルの保存](#sec_3_4)  
  [3.5 モデルの詳細確認](#sec_3_5)  

[4. モデルのデプロイ](#ch_4)  
  [4.1 モデルのデプロイ](#sec_4_1)  
  [4.2 デプロイメントの詳細確認](#sec_4_2)  

[5. 予測(Watsonライブラリを利用する方法)](#ch_5)   
  [5.1 入力用変数の組み立て](#sec_5_1)  
  [5.2 予測実施](#sec_5_2)  
  [5.3 結果確認](#sec_5_3)  

[6. 予測(Watsonライブラリを使わない方法](#ch_6)  
  [6.1 Token取得](#sec_6_1)  
  [6.2 Header組み立て](#sec_6_2)  
  [6.3 URL取得](#sec_6_3)  
  [6.4 入力用変数組み立て](#sec_6_4)  
  [6.5 予測実施](#sec_6_5)  
  [6.6 結果確認](#sec_6_6)


## バージョン確認
Pythonが3.8以上、scikit-learnが0.24以上になった場合、下の手順は見直す必要があります。

In [ ]:
import sys
print('Python: ', sys.version)

import sklearn
print('scikit-learn: ', sklearn.__version__)

<a id='ch_2'></a>
## 2. データロードと学習

<a id='sec_2_1'></a>
## 2.1 データのロード

In [ ]:
# IRIS Dataset を利用する

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()
X = iris.data
Y = iris.target
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,train_size=0.7, random_state=99)


In [ ]:
# 学習データの先頭10行

print(X_test[:10])
print(Y_test[:10])

<a id='sec_2_2'></a>
## 2.2 学習

In [ ]:
# アルゴリズムはRandom Forestを利用
# 作成後のモデルインスタンス変数名はmodel

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
model = clf.fit(X_train, Y_train)

<a id='ch_3'></a>
# 3. モデルの保存

<a id='sec_3_1'></a>
## 3.1 必要ライブラリの導入

In [ ]:
# 必要ライブラリの導入
!pip install -U ibm-watson-machine-learning | tail -n 1

<a id='sec_3_2'></a>
## 3.2 APIClient インスタンスの生成とデプロイメントスペースの関連付け

In [ ]:
# ロケーションの指定
# ロケーションは下記のコマンドの結果得られたものを使う
#
# $ ibmcloud resource service-instance

location = 'us-south'  # Dallas
#location = 'jp-tok'    # Tokyo
#location = 'au-syd'  # Sydney

# APIKeyの指定
# API key は次のリンク先から生成する
#
# https://cloud.ibm.com/iam/apikeys

apikey = "xxxx"

wml_credentials = {
    "apikey": apikey,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

In [ ]:
# API Clientインスタンスの生成

from ibm_watson_machine_learning import APIClient
client = APIClient(wml_credentials)

In [ ]:
# space_idの取得
client.spaces.list()

In [ ]:
# 上の結果を基に手で設定する
space_id = 'xxxx'

In [ ]:
# デプロイメントスペースIDの設定
client.set.default_space(space_id)

<a id='sec_3_3'></a>
## 3.3 Software Specification ID の取得

参考リンク:   
 https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_latest/wsj/wmls/wmls-deploy-python-types.html
    
Scikit-Learn 0.23 は``default_py3.7``となっているので、これを利用する。

In [ ]:
sofware_spec_uid = client.software_specifications.get_id_by_name("default_py3.7")
print(sofware_spec_uid)

<a id='sec_3_4'></a>
## 3.4 モデルの保存

モデルの保存はDeployment Spaceに対して行われる

In [ ]:
metadata = {
            client.repository.ModelMetaNames.NAME: 'Scikit IRIS random forest',                # モデル名称
            client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.23',                              # モデル種別 scikit-learn 0.23
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid # 上で取得したsofware_spec_uidを指定
}

published_model = client.repository.store_model(
    model=model,
    meta_props=metadata,
    training_data=X_train,
    training_target=Y_train)

<a id='sec_3_5'></a>
## 3.5 モデルの詳細確認

In [ ]:
import json

published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

In [ ]:
client.repository.list_models()

<a id='ch_4'></a>
# 4. モデルのデプロイ

<a id='sec_4_1'></a>
## 4.1 モデルのデプロイ
この例ではAPIを利用してモデルのデプロイを行っています。同じことはUIでも可能です。

In [ ]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Scikit-Learn Iris Model online",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)

In [ ]:
# デプロイメントの一覧表示

client.deployments.list()

In [ ]:
# depolyment_uid 取得

deployment_uid = client.deployments.get_uid(created_deployment)
print(deployment_uid)

<a id='sec_4_2'></a>
## 4.2 デプロイメントの詳細確認
``get_details``関数を使って、今新規に作ったデプロイメントの詳細を確認します。

In [ ]:
client.deployments.get_details(deployment_uid)

<a id='ch_5'></a>
# 5. 予測(Watsonライブラリを利用する方法)
予測に関してはWatsonライブラリを利用する方法と利用しない方法の2つを紹介します。  
5章で紹介するのは、Watsonライブラリを利用するパターンです。

<a id='sec_5_1'></a>
## 5.1 入力用変数の組み立て

In [ ]:
# 検証データの先頭N個で予測する
N = 20

# 予測用入力変数 scoring_payload の組み立て

# 先頭のN個を抽出し、リスト化する
values = X_test[:N].tolist()

# valuesを元にscoring_payload変数を設定
scoring_payload = {"input_data": [{"values": values}]}

<a id='sec_5_2'></a>
## 5.2 予測実施
depolymentと同時に予測をする場合は、変数deployment_uidがすでに設定されています。  
予測のみ別途行う場合は、deployment_uidの設定を手動で行います。  
deployment_uidは、デプロイメントの管理画面から取得可能です。  
あるいは、下記``client.deployments.list()``の結果を利用します(GUIDの値)。


In [ ]:
client.deployments.list()

In [ ]:
# 手動設定が必要な場合は、コメントをはずして下の行を有効にする
#  deployment_uid = 'xxxx'

# deployment_uidの確認
print(deployment_uid)

In [ ]:
# 予測の実施
predictions = client.deployments.score(deployment_uid, scoring_payload)

<a id='sec_5_3'></a>
## 5.3 結果確認

In [ ]:
print(predictions)

In [ ]:
# 予測値のみを抽出

import numpy as np
w1 = predictions['predictions'][0]['values']
w2 = np.array([x[0] for x in w1])

print(w2)

In [ ]:
# 正解値

print(Y_test[:N])

####  混同行列の表示

In [ ]:
import pandas as pd
from IPython.display import display

# 混同行列表示用関数

def make_cm(matrix, columns):
    # matrix numpy配列
    
    # columns 項目名リスト
    n = len(columns)
    
    # '正解データ'をn回繰り返すリスト生成
    act = ['正解データ'] * n
    pred = ['予測結果'] * n
    
    #データフレーム生成
    cm = pd.DataFrame(matrix, 
        columns=[pred, columns], index=[act, columns])
    return cm

In [ ]:
# 混同行列表示

from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(Y_test[:N], w2)

# make_cmを使った混同行列標示
cm = make_cm(matrix, iris.target_names)
display(cm)

<a id='ch_6'></a>
#  6. 予測(Watsonライブラリを使わない方法
6章では、Watsonライブラリを使わずに予測を行う方法を説明します。  
5章と比較すると多少手順は長くなりますが、基本的なライブラリしか使っていないため、汎用的な環境で予測できることが特徴です。

<a id='sec_6_1'></a>
## 6.1 Token取得

In [ ]:
import urllib3, requests, json

# トークン取得
# apikeyは3.2で使ったのと同じものを利用
url = "https://iam.cloud.ibm.com/identity/token"

headers = { "Content-Type" : "application/x-www-form-urlencoded",
          "Accept" : "application/json" }

data    = "apikey=" + apikey + "&grant_type=urn:ibm:params:oauth:grant-type:apikey"

response  = requests.post( url, headers=headers, data=data)
iam_token = response.json()["access_token"]
print('iam_token = ', iam_token)

<a id='sec_6_2'></a>
## 6.2 Header組み立て

In [ ]:
# headerの組み立て
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + iam_token}

<a id='sec_6_3'></a>
## 6.3 URL取得

In [ ]:
# URLの取得
scoring_url = client.deployments.get_scoring_href(created_deployment)

# scoring_urlはデプロイメント管理画面からも取得可能です。その場合は、下記のコメントをはずして直接指定します。
# scoring_url = "xxxx"

print(scoring_url)

In [ ]:
# バージョンの付加
# Watson ML v2の場合、バージョンをURLパラメータで追加することが必須です。

scoring_url2 = scoring_url + '?version=2020-09-01'

<a id='sec_6_4'></a>
## 6.4 入力用変数組み立て
このコードは5.1とまったく同じです。

In [ ]:
# scoring_payloadの組み立て

# 検証データの先頭N個で予測する
N = 20

# 予測用入力変数 scoring_payload の組み立て

# 先頭のN個を抽出し、リスト化する
values = X_test[:N].tolist()

# valuesを元にscoring_payload変数を設定
scoring_payload = {"input_data": [{"values": values}]}


<a id='ch_6_5'></a>
## 6.5 予測実施
Watson APIを使わず、標準的なライブラリ``requests``の``post``関数を利用しています。

In [ ]:
# 予測の実施
direct_scoring = requests.post(scoring_url2, json=scoring_payload, headers=header)

<a id='sec_6_6'></a>
## 6.6 結果確認

In [ ]:
# 結果確認 (リターンコード表示)

print(direct_scoring)

In [ ]:
# 結果確認　(戻り値表示)

print(direct_scoring.text)

In [ ]:
# 予測値のみを抽出

w0 = json.loads(direct_scoring.text)
w1 =w0 ['predictions'][0]['values']
w2 = np.array([x[0] for x in w1])

print(w2)